# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [3]:
from openai import OpenAI

client = OpenAI(
    # 中转Key或者直连Key
    api_key='sk-hXK719WAh86MxL6u2877B33101154f6cAcDdC89e1889F551',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://pic3.zhimg.com/v2-f307317a06f494b304a72481b8538475_r.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="这幅图展示了一张写有中文笔记的纸张，内容着重于英语语法学习。笔记分成几个部分：\n\n1. **一般现在时态** - 讲解了如何使用一般现在时态，列出了be动词的形态（be, am, is, are）、常规动词第三人称单数的变化（例如like - likes, run - runs），以及特殊的第三人称单数变化，如cry - cries, teach - teaches。\n\n2. **句型** - 开头提到了数个表达频率的副词（always, seldom）。接下来，笔记列出了英语中重要的连词（例如though, as, because, until）和其他辅助词，用以构建更加完整和复杂的句子。\n\n3. **简单运用** - 最后部分简要说明了如何将这些语法规则应用到日常句型中，如“我今天离开”构成“I'm leaving today”。\n\n整体上，这份笔记体现了学习者在英语语法规则上的", role='assistant', function_call=None, tool_calls=None))


In [4]:
response.choices[0].message.content

"这幅图展示了一张写有中文笔记的纸张，内容着重于英语语法学习。笔记分成几个部分：\n\n1. **一般现在时态** - 讲解了如何使用一般现在时态，列出了be动词的形态（be, am, is, are）、常规动词第三人称单数的变化（例如like - likes, run - runs），以及特殊的第三人称单数变化，如cry - cries, teach - teaches。\n\n2. **句型** - 开头提到了数个表达频率的副词（always, seldom）。接下来，笔记列出了英语中重要的连词（例如though, as, because, until）和其他辅助词，用以构建更加完整和复杂的句子。\n\n3. **简单运用** - 最后部分简要说明了如何将这些语法规则应用到日常句型中，如“我今天离开”构成“I'm leaving today”。\n\n整体上，这份笔记体现了学习者在英语语法规则上的"

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [6]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    # 中转Key或者直连Key
    api_key='sk-hXK719WAh86MxL6u2877B33101154f6cAcDdC89e1889F551',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [6]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图展示了1980年到2020年间，美国、中国、日本和德国的国内生产总值（GDP）比较。每个国家的GDP以万亿美元为单位标出。从图中可以看出：

- **美国（蓝线）**：GDP持续增长，从1980年的约3万亿美金增长到2020年的超过20万亿美金。
- **中国（红线）**：自1980年起，GDP增长非常显著，从不到1万亿美金增长至接近15万亿美金。
- **日本（紫线）**：1980年至1995年GDP增长明显，之后增长放缓，并在2005年至2015年间保持相对平稳。
- **德国（绿线）**：GDP增长较为平稳，从1980年的不到1万亿美金增长至2020年的约4万亿美金。

总体而言，图表清晰地表示了这四个经济体在过去四十年间的经济增长情况，其中中国的增长尤其引人注目，显示了其经济的迅速崛起。美国则保持了其全球经济领导地位的增长趋势。日本在90年代后增长放缓，而德国则显示出稳定的增长模式。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [7]:
content = query_base64_image_description("./images/gpt-4v.jpg")
print(content)

ConnectTimeout: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025A7C4B0C70>, 'Connection to api.openai.com timed out. (connect timeout=None)'))

#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [ ]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [ ]:
from openai import OpenAI

client = OpenAI(
    # 中转Key或者直连Key
    api_key='sk-hXK719WAh86MxL6u2877B33101154f6cAcDdC89e1889F551',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://pic3.zhimg.com/v2-f307317a06f494b304a72481b8538475_r.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

In [1]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    # 中转Key或者直连Key
    api_key='sk-hXK719WAh86MxL6u2877B33101154f6cAcDdC89e1889F551',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

In [2]:
content = query_base64_image_description("./images/gpt-4v.jpg")
print(content)

这张图显示的是一份中文的英语语法笔记。主要内容包括：

1. **动词形式的基础规则**：
   - 如何使用基本的动词形式，包括一些不规则的变化如“be (am/is/are)”，以及第三人称单数的变化规则（如："like - likes", "run - runs"等）。

2. **动词的时态**：
    - 对相关时态的使用做了说明，包括一般现在时的构成和用法。
    - 提到了几种基本的时态，如一般过去时、现在进行时等。

3. **常用词汇和短语**：
    - 解释了常用的频率副词（如"always", "seldom"）和链接词（如"when", "before", "although"等）。

4. **基础句式的构成**：
    - 讲述了简单句子的构建方式，特别是使用动词和介词。

这些笔记的目的是帮助学习者掌握英语语法的基础知识，适合英语学习初期的学生使用。整体上排版清晰，条理分明，适合快速查阅和复习。


In [ ]:
from IPython.display import display, Markdown
content = query_base64_image_description("./images/gpt-4v.jpg")
display(Markdown(content))